In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# ✅ Step 1: Initialize Spark Session
spark = SparkSession.builder.appName("TitanicMLPipeline").getOrCreate()

# ✅ Step 2: Create In-Memory Dataset (Titanic Data Sample)
data = [
    (1, "male", 22, 1, 0, "S", 0),   # Did not survive
    (2, "female", 38, 1, 0, "C", 1),  # Survived
    (3, "female", 26, 0, 0, "S", 1),  # Survived
    (4, "male", 35, 0, 0, "S", 0),   # Did not survive
    (5, "male", 28, 0, 0, "C", 0),   # Did not survive
    (6, "female", 40, 1, 1, "Q", 1),  # Survived
]

columns = ["PassengerId", "Sex", "Age", "SibSp", "Parch", "Embarked", "Survived"]
df = spark.createDataFrame(data, columns)

# ✅ Step 3: Feature Engineering (Convert Categorical Data)
# Convert "Sex" column to numeric
indexer_sex = StringIndexer(inputCol="Sex", outputCol="SexIndexed")
df = indexer_sex.fit(df).transform(df)

# Convert "Embarked" column to numeric
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="EmbarkedIndexed")
df = indexer_embarked.fit(df).transform(df)

# ✅ Step 4: Prepare Features for Model Training
feature_cols = ["Age", "SibSp", "Parch", "SexIndexed", "EmbarkedIndexed"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Select final columns
df = df.select("features", col("Survived").alias("label"))

# ✅ Step 5: Split Data into Train & Test Sets
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

# ✅ Step 6: Train Logistic Regression Model
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

# ✅ Step 7: Make Predictions
predictions = model.transform(test_df)
predictions.select("features", "label", "prediction").show()

# ✅ Step 8: Evaluate Model Performance
evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Model AUC: {auc:.3f}")

# ✅ Step 9: Stop Spark Session
spark.stop()


25/02/13 10:52:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/13 10:52:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/13 10:52:37 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
25/02/13 10:53:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/02/13 10:53:08 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[38.0,1.0,0.0,0.0...|    1|       1.0|
|(5,[0,3],[35.0,1.0])|    0|       0.0|
+--------------------+-----+----------+

Model AUC: 1.000


In [ ]:
IMP: High AUC( Area Under Curve) means the Model performed well